In [1]:
import datetime
import weave
weave.use_frontend_devmode()
import time
from weave.ecosystem import shawn

In [2]:
model = shawn.eval.Model("a")
weave.save(model, 'model-a')
proc = shawn.eval.PredictionProcess("000", model, [])
proc.predictions.append(shawn.eval.Prediction(datetime.datetime(2022, 10, 8, 10, 26), 0, 3))
proc.predictions.append(shawn.eval.Prediction(datetime.datetime(2022, 10, 8, 10, 31), 1, 4))
proc.predictions.append(shawn.eval.Prediction(datetime.datetime(2022, 10, 8, 10, 35), 2, 5))
proc.predictions.append(shawn.eval.Prediction(datetime.datetime(2022, 10, 8, 10, 40), 3, 5))
proc.predictions.append(shawn.eval.Prediction(datetime.datetime(2022, 10, 8, 10, 42), 2, 4))
proc.predictions.append(shawn.eval.Prediction(datetime.datetime(2022, 10, 8, 10, 44), 2, 5))
weave.save(proc, name="proc-a-1")

model = shawn.eval.Model("b")
weave.save(model, 'model-b')
proc = shawn.eval.PredictionProcess("000", model, [])
proc.predictions.append(shawn.eval.Prediction(datetime.datetime(2022, 10, 8, 10, 26), 3, 4))
proc.predictions.append(shawn.eval.Prediction(datetime.datetime(2022, 10, 8, 10, 31), 2, 2))
proc.predictions.append(shawn.eval.Prediction(datetime.datetime(2022, 10, 8, 10, 35), 0, 6))
proc.predictions.append(shawn.eval.Prediction(datetime.datetime(2022, 10, 8, 10, 40), 4, 4))
proc.predictions.append(shawn.eval.Prediction(datetime.datetime(2022, 10, 8, 10, 42), 0, 2))
proc.predictions.append(shawn.eval.Prediction(datetime.datetime(2022, 10, 8, 10, 44), 9, 2))
weave.save(proc, name="proc-b-1")
pass

In [3]:
models = weave.ops.objects(shawn.eval.Model.WeaveType(), 'latest', int(time.time()))
weave.use(models)

[<LocalArtifactRef(5262427232) artifact=<LocalArtifact(5262426752) model-b db6d55f9737c8e7dd0f1fee43f28ab13> path=_obj type=ModelType() obj=False extra=None,
 <LocalArtifactRef(5262428144) artifact=<LocalArtifact(5262426800) model-a e05cd43f15469cb9fe2ae3b228890947> path=_obj type=ModelType() obj=False extra=None]

In [4]:
panel = weave.panels.Group2(
    preferHorizontal=True,
    items={
        "sidebar": weave.panels.Group2(
            style="width: 200px; padding: 16px;",
            items={
                "model": weave.panels.ObjectPicker(models, label='model')
            }
        ),
        "main": lambda sidebar: weave.panels.Group2(
            items={
                "plots": weave.panels.Group2(
                    style="height: 300px;",
                    preferHorizontal=True,
                    equalSize=True,
                    items={
                        "input_plot": weave.panels.LabeledItem(
                            label="Input",
                            item=weave.panels.Plot(
                                sidebar.config.items['model'].config.choice.get().all_predictions(
                                    datetime.datetime(2022, 1, 1), datetime.datetime(2023, 1, 1)
                                ),
                                x=lambda pred: pred.timestamp,
                                y=lambda pred: pred.input,
                                mark='line'
                            )
                        ),
                        "output_plot": weave.panels.LabeledItem(
                            label="Output",
                            item=weave.panels.Plot(
                                sidebar.config.items['model'].config.choice.get().all_predictions(
                                    datetime.datetime(2022, 1, 1), datetime.datetime(2023, 1, 1)
                                ),
                                x=lambda pred: pred.timestamp,
                                y=lambda pred: pred.output,
                                mark='line'
                            )
                        )
                    }
                ),
                "tables": weave.panels.Group2(
                    style="height: 300px;",
                    preferHorizontal=True,
                    equalSize=True,
                    items={
                        "predictions_table": weave.panels.LabeledItem(
                            label="Predictions Table",
                            item=weave.panels.Table(
                                sidebar.config.items['model'].config.choice.get().all_predictions(
                                    datetime.datetime(2022, 1, 1), datetime.datetime(2023, 1, 1)
                                ),
                                columns=[
                                    lambda pred: pred.timestamp,
                                    lambda pred: pred.input,
                                    lambda pred: pred.output
                                ])
                        )
                    }),
            }
        )
    })
from weave import storage
from rich import print
#print(storage.to_python(panel)['_val'])
ref = weave.save(panel, name='my-new-panel')
ref